In [2]:
from ultralytics import YOLO
import torch
import torch.nn as nn
import torch.nn.functional as F


def autopad(k, p=None):
    return k // 2 if p is None else p


class Conv(nn.Module):
    def __init__(self, c1, c2, k=3, s=1, p=None):
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p), bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = nn.ReLU6(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))


class DepthHead(nn.Module):
    """Depth head with multi-scale inverse depth output (1 channel per head)"""

    def __init__(self, ch=()):
        super().__init__()
        self.nl = len(ch)  # number of input feature maps
        self.stride = torch.zeros(self.nl)

        c2 = max(16, ch[0] // 4)  # intermediate channels

        # 创建每层的 Conv 结构（Conv → Conv → 1x1 输出 1 通道）
        self.cv2 = nn.ModuleList([Conv(c, c2, 3) for c in ch])
        self.cv3 = nn.ModuleList([Conv(c2, c2, 3) for _ in ch])
        self.cv4 = nn.ModuleList([nn.Conv2d(c2, 1, 1) for _ in ch])  # 每层输出 1 通道

    def forward(self, x):
        """x: list of feature maps"""
        out = [None] * self.nl
        for i in range(self.nl):
            if i ==0:
                feat = self.cv2[i](x[i])
                feat = self.cv3[i](feat)
                up_feat = F.interpolate(feat, size=x[i + 1].shape[2:], mode='nearest')
                x[i + 1] = self.cv2[i+1](x[i + 1]) + up_feat
            elif i == self.nl - 1:
                feat = self.cv3[i](feat)
            else:
                feat = self.cv3[i](x[i])
                up_feat = F.interpolate(feat, size=x[i + 1].shape[2:], mode='nearest')
                x[i + 1] = self.cv2[i+1](x[i + 1]) + up_feat

            depth = self.cv4[i](feat)        # 输出为 B×1×H×W
            inv_depth = 1.0 / (depth + 1e-6)  # 计算 inverse depth
            out[i] = inv_depth

        return out



In [1]:
model = DepthHead(ch=[64, 64, 64])
x = [torch.randn(1, 64, 80, 80),
     torch.randn(1, 64, 40, 40),
     torch.randn(1, 64, 20, 20)]

y = model(x)
for d in y:
    print(d.shape)  # → torch.Size([1, 1, H, W])

NameError: name 'DepthHead' is not defined

In [ ]:
import cv2
import numpy as np
img = np.zeros((300, 300, 3), dtype=np.uint8)
cv2.imshow("test", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
